In [1]:
import numpy as np
from numpy import ndarray

**Problem Statement 1**

You are starting a company that grows and sells wild mushrooms. Since not all mushrooms are edible, you'd like to be able to tell whether a given mushroom is edible or poisonous based on it's physical attributes. You have some existing data that you can use for this task:

|                                                             | Cap Color | Stalk Shape | Solitary | Edible |
|:-----------------------------------------------------------:|:---------:|:-----------:|:--------:|:------:|
| <img src="images/mushroom_0.png" alt="drawing" width="50"/> |   Brown   |   Tapering  |    Yes   |    1   |
| <img src="images/mushroom_1.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    Yes   |    1   |
| <img src="images/mushroom_2.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    No    |    0   |
| <img src="images/mushroom_3.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    No    |    0   |
| <img src="images/mushroom_4.png" alt="drawing" width="50"/> |   Brown   |   Tapering  |    Yes   |    1   |
| <img src="images/mushroom_5.png" alt="drawing" width="50"/> |    Red    |   Tapering  |    Yes   |    0   |
| <img src="images/mushroom_6.png" alt="drawing" width="50"/> |    Red    |  Enlarging  |    No    |    0   |
| <img src="images/mushroom_7.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    Yes   |    1   |
| <img src="images/mushroom_8.png" alt="drawing" width="50"/> |    Red    |   Tapering  |    No    |    1   |
| <img src="images/mushroom_9.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    No    |    0   |

For ease of implemententation we use one-hot encoding of the features.

In [2]:
X_train = np.array([
    [1, 1, 1], [1, 0, 1], [1, 0, 0], [1, 0, 0], [1, 1, 1],
    [0, 1, 1], [0, 0, 0], [1, 0, 1], [0, 1, 0], [1, 0, 0]
])
y_train = np.array([1, 1, 0, 0, 1, 0, 0, 1, 1, 0])
assert X_train.shape == (10, 3)
assert y_train.shape == (10,)

Now we build a decision tree with a stopping criteria of maximum depth of 2. To begin we start by creating a function to calculate the entropy for a given node using the formula:

$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$

In [3]:
def compute_entropy(y: ndarray) -> float:
    """
    Computes the entropy for a node.

    Args:
        y (ndarray): Numpy array indicating whether each example at a node is
            dible (`1`) or poisonous (`0`)

    Returns:
        entropy (float): Entropy at that node.

    """
    m = len(y)
    if m == 0:
        return 0

    p1 = np.sum(y) / m
    if (p1 == 1) or (p1 == 0):
        return 0

    p0 = 1 - p1
    entropy = (-p1 * np.log2(p1)) - (p0 * np.log2(p0))
    return entropy

assert compute_entropy(np.array([1] * 10)) == 0
assert compute_entropy(np.array([0] * 10)) == 0
assert compute_entropy(np.array([0] * 12 + [1] * 12)) == 1
y = np.array([1, 0, 1, 0, 1, 1, 1, 0, 1])
assert np.isclose(compute_entropy(-y + 1), compute_entropy(y), atol=1e-6)

Next, we write a helper function called `split_dataset` that takes in the data at a node and a feature to split on and splits it into left and right branches.
    

    

In [4]:
def split_dataset(
    X: ndarray, node_indices: list[int], feature: int
) -> tuple[list[int], list[int]]:
    """
    Splits the data at the given node into left and right branches.


    Args:
        X (ndarray): Data matrix of shape(n_samples, n_features).
        node_indices (list[int]): List containing the active indices.
            I.e, the samples being considered at this step.
        feature (int): Index of feature to split on.

    Returns:
        tuple[list[int], list[int]]: Indices with feature == 1, and == 0.
    """


    # You need to return the following variables correctly
    left_indices = []
    right_indices = []

    for index in node_indices:
        if X[index][feature] == 1:
            left_indices.append(index)
        else:
            right_indices.append(index)

    return left_indices, right_indices

assert split_dataset(X_train, list(range(10)), 0) == ([0, 1, 2, 3, 4, 7, 9], [5, 6, 8])
assert split_dataset(X_train, [0, 2, 4, 6, 8], 0) == ([0, 2, 4], [6, 8])

Next, we write a function called `compute_information_gain()` that takes in the training data, the indices at a node and a feature to split on and returns the information gain from the split:


$$\text{Information Gain} = H(p_1^\text{node})- (w^{\text{left}}H(p_1^\text{left}) + w^{\text{right}}H(p_1^\text{right}))$$

In [5]:
def compute_information_gain(
    X: ndarray, y: ndarray, node_indices: ndarray, feature: int
) -> float:
    """
    Compute the information of splitting the node on a given feature.

    Args:
        X (ndarray): Data matrix of shape(n_samples, n_features).
        y (ndarray): Array with n_samples containing the target variable.
        node_indices (ndarray): List containing the active indices.
            I.e, the samples being considered in this step.

    Returns:
        cost (float): Cost computed.
    """
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    m = len(X[node_indices])
    root_entropy = compute_entropy(y[node_indices])
    w_left = len(left_indices) / m
    w_right = len(right_indices) / m
    entropy_left = compute_entropy(y[left_indices])
    entropy_right = compute_entropy(y[right_indices])
    weighted_entropy = (w_left * entropy_left) + (w_right * entropy_right)
    information_gain = root_entropy - weighted_entropy
    return information_gain

X_cig = np.array([[1, 0], [1, 0], [1, 0], [0, 0], [0, 1]])
y_cig = np.array([[0, 1, 0, 1, 0]]).T

assert compute_information_gain(
    X_cig, np.array([[0, 0, 0, 0, 0]]).T, list(range(5)), 0) == 0
assert np.isclose(compute_information_gain(
    X_cig, y_cig, list(range(5)), 0), 0.019973, atol=1e-6)
assert np.isclose(compute_information_gain(
    X_cig, y_cig, list(range(5)), 1), 0.170951, atol=1e-6)
assert np.isclose(compute_information_gain(
    X_cig, y_cig, list(range(4)), 0), 0.311278, atol=1e-6)

Now we write a function to get the best feature to split on by computing the information gain from each feature as we did above and returning the feature that gives the maximum information gain

In [6]:
def get_best_split(X: ndarray, y: ndarray, node_indices: ndarray) -> int:
    """
    Returns the optimal feature to split the node data.

    Args:
        X (ndarray): Data matrix of shape(n_samples, n_features)
        y (ndarray): Array with n_samples containing the target variable.
        node_indices (ndarray): List containing the active indices.
            I.e, the samples being considered in this step.

    Returns:
        best_feature (int): The index of the best feature to split.
    """

    best_feature = -1
    best_information_gain = 0

    for feature in range(X.shape[1]):
        information_gain = compute_information_gain(X, y, node_indices, feature)
        if information_gain > best_information_gain:
            best_information_gain = information_gain
            best_feature = feature
    return best_feature

X = np.array([[1, 0], [1, 0], [1, 0], [0, 0], [0, 1]])
y = np.array([[0, 0, 0, 0, 0]]).T
node_indexes = list(range(5))
assert get_best_split(X, y, node_indexes) == -1
assert get_best_split(X, X[:, 0], node_indexes) == 0
assert get_best_split(X, X[:, 1], node_indexes) == 1
assert get_best_split(X, 1 - X[:, 0], node_indexes) == 0
assert get_best_split(X, np.array([[0, 1, 0, 1, 0]]).T, node_indexes) == 1
assert get_best_split(X, np.array([[0, 1, 0, 1, 0]]).T, [2, 3, 4]) == 0

Finally, we use the functions implemented above to generate a decision tree by successively picking the best feature to split on until we reach the stopping criteria (maximum depth is 2).

In [8]:
def build_tree_recursive(
    X: ndarray, y: ndarray, node_indices: ndarray,
    branch_name: str, max_depth: int, current_depth: int
) -> None:
    """
    Build a tree using the recursive algorithm that split the dataset into
    2 subgroups at each node. This function just prints the tree.

    Args:
        X (ndarray): Data matrix of shape(n_samples, n_features)
        y (ndarray): Array with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices.
            I.e, the samples being considered in this step.
        branch_name (str): Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int): Max depth of the resulting tree.
        current_depth (int): Current depth. Used during recursive call.
    """

    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return

    # Otherwise, get best split and split the data
    best_feature = get_best_split(X, y, node_indices)

    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))

    # Split the dataset at the best feature
    l_indices, r_indices = split_dataset(X, node_indices, best_feature)
    tree.append((l_indices, r_indices, best_feature))

    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, l_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, r_indices, "Right", max_depth, current_depth+1)

tree = []
build_tree_recursive(X_train, y_train, list(range(10)), "Root", 2, 0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]
